In [23]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split
import glob

In [24]:
keras.__version__

'2.2.4-tf'

In [25]:
path = glob.glob("C:/Users/suhas/Documents/Mini Project/Leaves/*.jpg")

In [26]:
IMAGE_SIZE = 100
num_classes = 32
epochs = 100

In [27]:
def label(img_name):
    num = int(img_name[-8:-4])
    if num >= 1001 and num <= 1059:
        return 0 #'pubescent bamboo'
    elif num >= 1060 and num <= 1122:
        return 1 #'chinese horse chestnut'
    elif num >= 1552 and num <= 1616:
        return 2 #'anhui barberry'
    elif num >= 1123 and num <= 1194:
        return 3 #'chinese redbud'
    elif num >= 1195 and num <= 1267:
        return 4 #'true indigo'
    elif num >= 1268 and num <= 1323:
        return 5 #'japanese maple'
    elif num >= 1324 and num <= 1385:
        return 6 #'nanmu'
    elif num >= 1386 and num <= 1437:
        return 7 #'castor aralia'
    elif num >= 1497 and num <= 1551:
        return 8 #'chinese cinnamon'
    elif num >= 1438 and num <= 1496:
        return 9 #'goldenrain tree'
    elif num >= 2001 and num <= 2050:
        return 10 #'big-fruited holly'
    elif num >= 2051 and num <= 2113:
        return 11 #'japanese cheesewood'
    elif num >= 2114 and num <= 2165:
        return 12 #'wintersweet'
    elif num >= 2166 and num <= 2230:
        return 13 #'camphor tree'
    elif num >= 2231 and num <= 2290:
        return 14 #'japan arrowwood'
    elif num >= 2291 and num <= 2346:
        return 15 #'sweet osmanthus'
    elif num >= 2347 and num <= 2423:
        return 16 #'deodar'
    elif num >= 2424 and num <= 2485:
        return 17 #'gingko'
    elif num >= 2486 and num <= 2546:
        return 18 #'crepe myrtle'
    elif num >= 2547 and num <= 2612:
        return 19 #'oleander'
    elif num >= 2616 and num <= 2675:
        return 20 #'yew plum pine'
    elif num >= 3001 and num <= 3055:
        return 21 #'japanese flowering cherry'
    elif num >= 3056 and num <= 3110:
        return 22 #'glossy privet'
    elif num >= 3111 and num <= 3175:
        return 23 #'chinese toon'
    elif num >= 3176 and num <= 3229:
        return 24 #'peach'
    elif num >= 3230 and num <= 3281:
        return 25 #'ford woodlotus'
    elif num >= 3282 and num <= 3334:
        return 26 #'trident maple'
    elif num >= 3335 and num <= 3389:
        return 27 #'beales barberry'
    elif num >= 3390 and num <= 3446:
        return 28 #'southern magnolia'
    elif num >= 3447 and num <= 3510:
        return 29 #'canadian poplar'
    elif num >= 3511 and num <= 3563:
        return 30 #'chinese tulip tree'
    elif num >= 3566 and num <= 3621:
        return 31 #'tangerine'

# Creating Dataset

In [28]:
data = []
for image in path:
    leaf = label(image)
    img = cv2.imread(image)
    
    # Basic processing
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    imgBlur = cv2.GaussianBlur(imgGray, (3, 3), 0)
    thresh, imgBW = cv2.threshold(imgBlur, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    imgInv = cv2.bitwise_not(imgBW)
    kernel = np.ones((50, 50))
    imgClosed = cv2.morphologyEx(imgInv, cv2.MORPH_CLOSE, kernel)
    
    # Resize
    new = cv2.resize(imgClosed, (IMAGE_SIZE, IMAGE_SIZE))
    #Adding third dimension to shape
    new.shape += (1,)
    data.append([new, leaf])

In [29]:
len(data)

1907

In [30]:
X = np.array([d[0] for d in data])
len(X)

1907

In [31]:
X = np.divide(X, 255)
len(X)
X[0].shape

(100, 100, 1)

In [32]:
y = [d[1] for d in data]
len(y)

1907

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

# CNN

In [34]:
model = Sequential([
    layers.InputLayer(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPool2D(),
    # Dropout layer for regularisation
    layers.Dropout(rate=0.2),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPool2D(),
    # Dropout layer for regularisation
    layers.Dropout(rate=0.2),
    layers.Conv2D(64, 5, padding='same', activation='relu'),
    layers.MaxPool2D(),
    # Flattening output to connect to Densely Connected Layers
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [35]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [36]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 100, 100, 16)      160       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 25, 64)        51264     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 12, 64)       

In [37]:
history = model.fit(x=X_train, y=y_train, epochs=epochs, validation_data = (X_test, y_test))

Train on 1430 samples, validate on 477 samples
Epoch 1/100
1430/1430 [==============================] - 7s 5ms/sample - loss: 3.3884 - acc: 0.1308 - val_loss: 3.3794 - val_acc: 0.1384
Epoch 2/100
1430/1430 [==============================] - 7s 5ms/sample - loss: 3.3713 - acc: 0.1455 - val_loss: 3.3786 - val_acc: 0.1363
Epoch 3/100
1430/1430 [==============================] - 8s 5ms/sample - loss: 3.3396 - acc: 0.1783 - val_loss: 3.3600 - val_acc: 0.1530
Epoch 4/100
1430/1430 [==============================] - 8s 5ms/sample - loss: 3.3166 - acc: 0.2000 - val_loss: 3.3084 - val_acc: 0.2034
Epoch 5/100
1430/1430 [==============================] - 7s 5ms/sample - loss: 3.2176 - acc: 0.3021 - val_loss: 3.1671 - val_acc: 0.3522
Epoch 6/100
1430/1430 [==============================] - 7s 5ms/sample - loss: 3.1550 - acc: 0.3636 - val_loss: 3.1602 - val_acc: 0.3627
Epoch 7/100
1430/1430 [==============================] - 7s 5ms/sample - loss: 3.1321 - acc: 0.3853 - val_loss: 3.1232 - val_acc: 0

Epoch 60/100
1430/1430 [==============================] - 8s 6ms/sample - loss: 2.8759 - acc: 0.6420 - val_loss: 2.9337 - val_acc: 0.5849
Epoch 61/100
1430/1430 [==============================] - 8s 6ms/sample - loss: 2.8689 - acc: 0.6490 - val_loss: 2.9422 - val_acc: 0.5765
Epoch 62/100
1430/1430 [==============================] - 8s 6ms/sample - loss: 2.8683 - acc: 0.6497 - val_loss: 2.9309 - val_acc: 0.5870
Epoch 63/100
1430/1430 [==============================] - 8s 6ms/sample - loss: 2.8734 - acc: 0.6434 - val_loss: 2.9318 - val_acc: 0.5870
Epoch 64/100
1430/1430 [==============================] - 8s 6ms/sample - loss: 2.8638 - acc: 0.6531 - val_loss: 2.9254 - val_acc: 0.5912
Epoch 65/100
1430/1430 [==============================] - 8s 6ms/sample - loss: 2.8600 - acc: 0.6587 - val_loss: 2.9477 - val_acc: 0.5702
Epoch 66/100
1430/1430 [==============================] - 8s 6ms/sample - loss: 2.8572 - acc: 0.6608 - val_loss: 2.9337 - val_acc: 0.5849
Epoch 67/100
1430/1430 [==========